# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
df = pd.read_csv(os.path.join("..", "output_data", "cities.csv"))
df = df[["City","Lat","Lng","Max Temp (degrees F)","Humidity","Cloudiness","Wind Speed","Country","Date"]]
display(df)

,City,Lat,Lng,Max Temp (degrees F),Humidity,Cloudiness,Wind Speed,Country,Date
0,bilibino,68.0546,166.4372,-28.012,79,1,2.38,RU,1612045050
1,kapaa,22.0752,-159.3190,78.800,65,75,7.20,US,1612045050
2,klaksvik,62.2266,-6.5890,33.800,80,75,7.20,FO,1612045050
3,albany,42.6001,-73.9662,19.004,54,13,1.79,US,1612045050
4,walvis bay,-22.9575,14.5053,66.992,82,0,2.06,NaN,1612045050
...,...,...,...,...,...,...,...,...,...
554,mayo,38.8876,-76.5119,35.996,48,1,3.60,US,1612045139
555,abu samrah,35.3029,37.1841,49.712,67,98,5.88,SY,1612045140
556,antigonish,45.6169,-61.9986,24.998,79,100,1.79,CA,1612045005
557,kungurtug,50.5994,97.5228,-3.424,84,86,1.91,RU,1612045140


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)
locations = df[["Lat", "Lng"]]
humidity = df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 10, point_radius = 1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_temp_df = df[(df["Max Temp (degrees F)"] >= 71) & (df["Max Temp (degrees F)"]<76)]
ideal_wind_temp_df = ideal_temp_df[ideal_temp_df["Wind Speed"] < 10]
ideal_df = ideal_wind_temp_df[ideal_wind_temp_df["Cloudiness"] == 0]

ideal_df["Hotel Name"] = ""
display(ideal_df)

<ipython-input-8-e134844b4263>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df["Hotel Name"] = ""


,City,Lat,Lng,Max Temp (degrees F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,pisco,-13.7000,-76.2167,73.994,73,0,8.75,PE,1612045054,
53,lima,-12.0432,-77.0282,73.994,73,0,6.17,PE,1612044459,
88,muriwai beach,-36.8167,174.4500,73.994,42,0,1.64,NZ,1612045064,
91,mogadishu,2.0371,45.3438,75.380,82,0,9.12,SO,1612045064,
146,kaeo,-35.1000,173.7833,71.006,62,0,4.02,NZ,1612045073,
163,chilca,-12.5211,-76.7372,75.002,73,0,6.17,PE,1612045076,
189,dawei,14.0833,98.2000,71.006,65,0,1.68,MM,1612045081,
234,aden,12.7794,45.0367,74.372,72,0,3.82,YE,1612045088,
257,nouadhibou,20.9310,-17.0347,73.400,40,0,7.72,MR,1612045092,
279,taoudenni,22.6783,-3.9836,72.032,21,0,4.37,ML,1612045096,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotels = []
params = ({
    "key" : g_key,
})
for index, row in ideal_df.iterrows():
    city = ideal_df["City"][index]
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f'{lat},{lng}'
    params["radius"] = 5000
    params["keyword"] = "hotel"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params = params)
    data_json = response.json()
    
    try:
        hotels.append(data_json["results"][0]["name"])
        print(f'Seraching {city}. Hotel Found!')
    except:
        print(f'No hotels in {city} available. Try Airbnb.')
        hotels.append("NaN")
        
ideal_df["Hotel Name"] = hotels
hotel_df = ideal_df
display(hotel_df)

Seraching pisco. Hotel Found!
Seraching lima. Hotel Found!
Seraching muriwai beach. Hotel Found!
Seraching mogadishu. Hotel Found!
Seraching kaeo. Hotel Found!
Seraching chilca. Hotel Found!
Seraching dawei. Hotel Found!
Seraching aden. Hotel Found!
Seraching nouadhibou. Hotel Found!
No hotels in taoudenni available. Try Airbnb.
Seraching kwinana. Hotel Found!
Seraching valdivia. Hotel Found!
Seraching de aar. Hotel Found!
No hotels in nagapattinam available. Try Airbnb.
Seraching vila do maio. Hotel Found!


<ipython-input-12-59a724e82961>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df["Hotel Name"] = hotels


,City,Lat,Lng,Max Temp (degrees F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
23,pisco,-13.7000,-76.2167,73.994,73,0,8.75,PE,1612045054,HOSTAL MIRAMAR
53,lima,-12.0432,-77.0282,73.994,73,0,6.17,PE,1612044459,Hotel Carrera
88,muriwai beach,-36.8167,174.4500,73.994,42,0,1.64,NZ,1612045064,216 Luxury Accommodation Auckland
91,mogadishu,2.0371,45.3438,75.380,82,0,9.12,SO,1612045064,Dolphin Hotel
146,kaeo,-35.1000,173.7833,71.006,62,0,4.02,NZ,1612045073,Pacific Harbour Lodge
163,chilca,-12.5211,-76.7372,75.002,73,0,6.17,PE,1612045076,LA CASA DE DON JOAQUIN
189,dawei,14.0833,98.2000,71.006,65,0,1.68,MM,1612045081,The Mandolis Hotel and Restaurant
234,aden,12.7794,45.0367,74.372,72,0,3.82,YE,1612045088,Sheraton Gold Mohur Hotel
257,nouadhibou,20.9310,-17.0347,73.400,40,0,7.72,MR,1612045092,El Medina
279,taoudenni,22.6783,-3.9836,72.032,21,0,4.37,ML,1612045096,NaN


In [91]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [92]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))